In [1]:
from dice_game import DiceGame
import numpy as np

In [2]:
from abc import ABC, abstractmethod
from dice_game import DiceGame
import numpy as np


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass


class AlwaysHoldAgent(DiceGameAgent):
    def play(self, state):
        return (0, 1, 2)


class PerfectionistAgent(DiceGameAgent):
    def play(self, state):
        if state == (1, 1, 1) or state == (1, 1, 6):
            return (0, 1, 2)
        else:
            return ()
        
        
def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    
    while not game_over:
        action = agent.play(state)
        actions += 1
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score


In [3]:
"""Expected values policy"""
import time
import copy
import statistics

class Expected_Values(DiceGameAgent):
    def __init__(self, game):
        """
        if your code does any pre-processing on the game, you can do it here
        
        e.g. you could do the value iteration algorithm here once, store the policy, 
        and then use it in the play method
        
        you can always access the game with self.game
        """
        # this calls the superclass constructor (does self.game = game)
        super().__init__(game)
        #print(f"Dice: {self.game._dice}, Sides:{self.game._sides}, Values:{self.game._values}, Bias:{self.game._bias}, Penalty:{self.game._penalty}")

        self.reset()
        
    def reset(self):
        self.hold_all = tuple(_ for _ in range(self.game._dice))
        self.number_actions = 0
        self.action_expectation = {}
            
    def find_expected_value(self, state, action):
        rolls, game_over, self.rewards, probabilities = self.game.get_next_states(action, (state))
        
        possible_scores = []
        for roll in rolls:
            possible_scores.append(float(self.game.final_scores[roll] + (self.rewards * self.number_actions)))
                                       
        return sum(score * probability for score, probability in zip(possible_scores, probabilities))
        
    def play(self, state):
        """
        given a state, return the chosen action for this state
        at minimum you must support the basic rules: three six-sided fair dice
        
        if you want to support more rules, use the values inside self.game, e.g.
            the input state will be one of self.game.states
            you must return one of self.game.actions
        
        read the code in dicegame.py to learn more
        """
        #print(f"Dice: \t\t{state}")

        self.number_actions += 1
        for action in self.game.actions:
            # if expectation value for rolling all dice has already been calculated,
            # instead of running it again, just add on the reward
            if action == () and action in self.action_expectation.keys():
                self.action_expectation[action] += self.rewards
                continue
            if action == self.hold_all:
                # skip when action is to hold all dice
                continue
            expected_value = self.find_expected_value(state, action)
            self.action_expectation[action] = expected_value
        
        current_score = self.game.final_score(state) + (self.rewards * (self.number_actions - 1))
        
        self.action_expectation[self.hold_all] = current_score
        #print(f"actions: {self.number_actions}, current score: {current_score}\naction expectations: {self.action_expectation}\n\n")

        best_action = max(self.action_expectation, key = self.action_expectation.get)
        
        if best_action == self.hold_all:
            self.reset()
            
        return best_action
        

def main():
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    np.random.seed(7)
    
    game = DiceGame(dice=4, sides=3, values=[1, 2, 6], bias=[1, 0, 0], penalty=2)
    """agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")"""
    
    agent2 = Expected_Values(game)
    play_game_with_agent(agent2, game, verbose=True)
    

if __name__ == "__main__":
    main()

Testing agent: 
	Expected_Values
Starting dice: 
	(1, 1, 1, 1)

Action 1: 	(0, 1, 2, 3)

Final dice: (6, 6, 6, 6), score: 24


In [71]:
import time
import copy
import statistics
import math

class MyAgent(DiceGameAgent):
    def __init__(self, game):
        # this calls the superclass constructor (does self.game = game)
        super().__init__(game)
        print(f"Dice: {self.game._dice}, Sides:{self.game._sides}, Values:{self.game._values}, Bias:{self.game._bias}, Penalty:{self.game._penalty}")

        self.reset()
        
    def reset(self):
        self.hold_all = tuple(_ for _ in range(self.game._dice))
        self.number_actions = 0
        
        self.discount_rate = 0.99
        
        self.transition_reward = - self.game._penalty
        
        self.V = {}
        self.newV = {}
    
        self.Q_values = {}
        
        sample_roll = tuple(x+1 for x in self.hold_all)
        states, _, __, probabilities = self.game.get_next_states((), sample_roll)
        for state in states:
            self.V[state] = 0
        
            
    def Q(self, state, action):
        """Returns a utility value for a given state and action."""
        s_primes, _, __, probabilities = self.game.get_next_states(action, (state))
        possible_scores = []
        rewards = []
        self.Q_values[action] = 0
        for i in range(len(s_primes)):
            if action == self.hold_all:
                break # utility for holding all dice = 0
            s_prime = s_primes[i]
            possible_scores.append(float(self.game.final_scores[s_prime]))
            rewards.append(possible_scores[i] - self.current_score + (self.transition_reward * self.number_actions))
            self.Q_values[action] += probabilities[i] * (rewards[i] + (self.discount_rate * self.V[s_prime]))
        
    def play(self, state):
        """
        given a state, return the chosen action for this state
        at minimum you must support the basic rules: three six-sided fair dice
        
        if you want to support more rules, use the values inside self.game, e.g.
            the input state will be one of self.game.states
            you must return one of self.game.actions
        
        read the code in dicegame.py to learn more
        """
        #print(f"Dice: \t\t{state}")
        self.current_score = self.game.final_score(state) + (self.transition_reward * (self.number_actions))
        self.number_actions += 1
        
        self.V[state] = 0
        
        delta = math.inf # max relative change in the utility of any state
        while delta > 0.01:
            for action in self.game.actions:                    
                self.Q(state, action)

            best_action = max(self.Q_values, key = self.Q_values.get)
            self.newV[state] = self.Q_values[best_action]
            delta = abs(self.newV[state] - self.V[state])
            #print(f"Before - V: {self.V}, newV: {self.newV}")
            self.V[state] = self.newV[state]
            #print(f"After - V: {self.V}, newV: {self.newV}")
        
        #print()
        #print(best_action)
        return best_action
        
        

def main():
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    # np.random.seed(7)
    
    game = DiceGame()
    """agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")"""
    
    agent2 = MyAgent(game)
    play_game_with_agent(agent2, game, verbose=True)
    
    

if __name__ == "__main__":
    main()

Dice: 3, Sides:6, Values:[1 2 3 4 5 6], Bias:[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667], Penalty:1
Testing agent: 
	MyAgent
Starting dice: 
	(2, 4, 5)

Action 1: 	(0, 1, 2)

Final dice: (2, 4, 5), score: 11


In [80]:
import time
import copy
import statistics
import math

class MyAgent(DiceGameAgent):
    def __init__(self, game):
        # this calls the superclass constructor (does self.game = game)
        super().__init__(game)
        print(f"Dice: {self.game._dice}, Sides:{self.game._sides}, Values:{self.game._values}, Bias:{self.game._bias}, Penalty:{self.game._penalty}")

        self.hold_all = tuple(_ for _ in range(self.game._dice))
        self.number_actions = 0
        
        self.discount_rate = 1
        
        self.transition_reward = - self.game._penalty
        
        self.V = {}
        self.newV = {}
    
        self.Q_values = {}
        
        sample_roll = tuple(x+1 for x in self.hold_all)
        states, _, __, probabilities = self.game.get_next_states((), sample_roll)
        for state in states:
            self.V[state] = 0
        
            
    def Q(self, state, action):
        """Returns a utility value for a given state and action."""
        s_primes, _, __, probabilities = self.game.get_next_states(action, (state))
        possible_scores = []
        rewards = []
        self.Q_values[action] = 0
        for i in range(len(s_primes)):
            if action == self.hold_all:
                break # utility for holding all dice = 0
            s_prime = s_primes[i]
            possible_scores.append(float(self.game.final_scores[s_prime]))
            rewards.append(possible_scores[i] - self.current_score + (self.transition_reward * self.number_actions))
            self.Q_values[action] += probabilities[i] * (rewards[i] + (self.discount_rate * self.V[s_prime]))
        
    def play(self, state):
        """
        given a state, return the chosen action for this state
        at minimum you must support the basic rules: three six-sided fair dice
        
        if you want to support more rules, use the values inside self.game, e.g.
            the input state will be one of self.game.states
            you must return one of self.game.actions
        
        read the code in dicegame.py to learn more
        """
        #print(f"Dice: \t\t{state}")
        self.current_score = self.game.final_score(state) + (self.transition_reward * (self.number_actions))
        self.number_actions += 1
        
        self.V[state] = 0
        
        delta = math.inf # max relative change in the utility of any state
        while delta > 0.01:
            for action in self.game.actions:                    
                self.Q(state, action)

            best_action = max(self.Q_values, key = self.Q_values.get)
            self.newV[state] = self.Q_values[best_action]
            delta = abs(self.newV[state] - self.V[state])
            self.V[state] = self.newV[state]
        
        return best_action
        
        

def main():
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    # np.random.seed(7)
    
    game = DiceGame()
    """agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")"""
    
    agent2 = MyAgent(game)
    play_game_with_agent(agent2, game, verbose=True)
    
    

if __name__ == "__main__":
    main()

Dice: 3, Sides:6, Values:[1 2 3 4 5 6], Bias:[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667], Penalty:1
Testing agent: 
	MyAgent
Starting dice: 
	(4, 5, 6)

Action 1: 	(0, 1, 2)

Final dice: (4, 5, 6), score: 15


In [81]:
SKIP_TESTS = False

def tests():
    import time

    total_score = 0
    total_time = 0
    n = 10000

    np.random.seed(4)

    print("Testing basic rules.")
    print()

    game = DiceGame()

    start_time = time.process_time()
    test_agent = [Expected_Values(game), MyAgent(game)]
    total_time += time.process_time() - start_time
    low_scores = {}

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent[1], game)
        if score < 6:
            if score in low_scores.keys():
                low_scores[score] += 1
            else:
                low_scores[score] = 1
            
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Total time: {total_time:.4f} seconds")
    print(f"Low Scores: {low_scores}")
    
if not SKIP_TESTS:
    tests()

Testing basic rules.

Dice: 3, Sides:6, Values:[1 2 3 4 5 6], Bias:[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667], Penalty:1
Game 0 score: 12
Game 1 score: 11
Game 2 score: 11
Game 3 score: 17
Game 4 score: 14
Game 5 score: 8
Game 6 score: 16
Game 7 score: 11
Game 8 score: 11
Game 9 score: 12
Game 10 score: 10
Game 11 score: 15
Game 12 score: 13
Game 13 score: 11
Game 14 score: 11
Game 15 score: 11
Game 16 score: 15
Game 17 score: 12
Game 18 score: 17
Game 19 score: 12
Game 20 score: 12
Game 21 score: 12
Game 22 score: 14
Game 23 score: 11
Game 24 score: 11
Game 25 score: 13
Game 26 score: 14
Game 27 score: 13
Game 28 score: 11
Game 29 score: 10
Game 30 score: 16
Game 31 score: 10
Game 32 score: 17
Game 33 score: 12
Game 34 score: 11
Game 35 score: 14
Game 36 score: 17
Game 37 score: 12
Game 38 score: 12
Game 39 score: 11
Game 40 score: 11
Game 41 score: 16
Game 42 score: 9
Game 43 score: 15
Game 44 score: 15
Game 45 score: 13
Game 46 score: 15
Game 47 score: 14
Ga

Game 433 score: 10
Game 434 score: 13
Game 435 score: 13
Game 436 score: 15
Game 437 score: 17
Game 438 score: 14
Game 439 score: 16
Game 440 score: 14
Game 441 score: 14
Game 442 score: 13
Game 443 score: 13
Game 444 score: 11
Game 445 score: 11
Game 446 score: 8
Game 447 score: 14
Game 448 score: 15
Game 449 score: 14
Game 450 score: 14
Game 451 score: 13
Game 452 score: 13
Game 453 score: 11
Game 454 score: 13
Game 455 score: 13
Game 456 score: 12
Game 457 score: 12
Game 458 score: 15
Game 459 score: 17
Game 460 score: 15
Game 461 score: 14
Game 462 score: 14
Game 463 score: 18
Game 464 score: 12
Game 465 score: 14
Game 466 score: 13
Game 467 score: 12
Game 468 score: 17
Game 469 score: 15
Game 470 score: 12
Game 471 score: 13
Game 472 score: 6
Game 473 score: 13
Game 474 score: 14
Game 475 score: 14
Game 476 score: 12
Game 477 score: 9
Game 478 score: 17
Game 479 score: 13
Game 480 score: 11
Game 481 score: 13
Game 482 score: 15
Game 483 score: 12
Game 484 score: 14
Game 485 score:

Game 868 score: 12
Game 869 score: 14
Game 870 score: 17
Game 871 score: 16
Game 872 score: 12
Game 873 score: 16
Game 874 score: 11
Game 875 score: 13
Game 876 score: 15
Game 877 score: 16
Game 878 score: 13
Game 879 score: 15
Game 880 score: 13
Game 881 score: 16
Game 882 score: 14
Game 883 score: 13
Game 884 score: 12
Game 885 score: 13
Game 886 score: 11
Game 887 score: 18
Game 888 score: 13
Game 889 score: 10
Game 890 score: 9
Game 891 score: 14
Game 892 score: 13
Game 893 score: 16
Game 894 score: 8
Game 895 score: 15
Game 896 score: 13
Game 897 score: 13
Game 898 score: 17
Game 899 score: 3
Game 900 score: 14
Game 901 score: 15
Game 902 score: 13
Game 903 score: 13
Game 904 score: 13
Game 905 score: 13
Game 906 score: 15
Game 907 score: 13
Game 908 score: 14
Game 909 score: 13
Game 910 score: 13
Game 911 score: 13
Game 912 score: 12
Game 913 score: 13
Game 914 score: 7
Game 915 score: 18
Game 916 score: 14
Game 917 score: 13
Game 918 score: 11
Game 919 score: 15
Game 920 score: 

Game 1288 score: 8
Game 1289 score: 10
Game 1290 score: 15
Game 1291 score: 9
Game 1292 score: 14
Game 1293 score: 14
Game 1294 score: 13
Game 1295 score: 8
Game 1296 score: 14
Game 1297 score: 11
Game 1298 score: 15
Game 1299 score: 13
Game 1300 score: 15
Game 1301 score: 9
Game 1302 score: 15
Game 1303 score: 10
Game 1304 score: 12
Game 1305 score: 11
Game 1306 score: 16
Game 1307 score: 14
Game 1308 score: 7
Game 1309 score: 17
Game 1310 score: 16
Game 1311 score: 14
Game 1312 score: 14
Game 1313 score: 14
Game 1314 score: 15
Game 1315 score: 14
Game 1316 score: 16
Game 1317 score: 15
Game 1318 score: 12
Game 1319 score: 11
Game 1320 score: 13
Game 1321 score: 13
Game 1322 score: 14
Game 1323 score: 17
Game 1324 score: 14
Game 1325 score: 15
Game 1326 score: 15
Game 1327 score: 10
Game 1328 score: 12
Game 1329 score: 11
Game 1330 score: 13
Game 1331 score: 13
Game 1332 score: 13
Game 1333 score: 13
Game 1334 score: 14
Game 1335 score: 16
Game 1336 score: 12
Game 1337 score: 10
Game 

Game 1701 score: 13
Game 1702 score: 7
Game 1703 score: 17
Game 1704 score: 15
Game 1705 score: 13
Game 1706 score: 14
Game 1707 score: 11
Game 1708 score: 13
Game 1709 score: 15
Game 1710 score: 13
Game 1711 score: 13
Game 1712 score: 11
Game 1713 score: 8
Game 1714 score: 17
Game 1715 score: 16
Game 1716 score: 13
Game 1717 score: 14
Game 1718 score: 17
Game 1719 score: 13
Game 1720 score: 11
Game 1721 score: 10
Game 1722 score: 15
Game 1723 score: 12
Game 1724 score: 13
Game 1725 score: 16
Game 1726 score: 14
Game 1727 score: 10
Game 1728 score: 10
Game 1729 score: 16
Game 1730 score: 15
Game 1731 score: 12
Game 1732 score: 12
Game 1733 score: 10
Game 1734 score: 12
Game 1735 score: 5
Game 1736 score: 18
Game 1737 score: 12
Game 1738 score: 13
Game 1739 score: 15
Game 1740 score: 15
Game 1741 score: 14
Game 1742 score: 6
Game 1743 score: 11
Game 1744 score: 16
Game 1745 score: 13
Game 1746 score: 14
Game 1747 score: 12
Game 1748 score: 12
Game 1749 score: 14
Game 1750 score: 10
Game

...

Game 8326 score: 10
Game 8327 score: 13
Game 8328 score: 13
Game 8329 score: 15
Game 8330 score: 16
Game 8331 score: 16
Game 8332 score: 13
Game 8333 score: 13
Game 8334 score: 13
Game 8335 score: 9
Game 8336 score: 12
Game 8337 score: 7
Game 8338 score: 17
Game 8339 score: 11
Game 8340 score: 17
Game 8341 score: 12
Game 8342 score: 12
Game 8343 score: 13
Game 8344 score: 12
Game 8345 score: 14
Game 8346 score: 14
Game 8347 score: 18
Game 8348 score: 15
Game 8349 score: 15
Game 8350 score: 18
Game 8351 score: 14
Game 8352 score: 15
Game 8353 score: 13
Game 8354 score: 15
Game 8355 score: 15
Game 8356 score: 14
Game 8357 score: 11
Game 8358 score: 16
Game 8359 score: 14
Game 8360 score: 14
Game 8361 score: 14
Game 8362 score: 14
Game 8363 score: 7
Game 8364 score: 15
Game 8365 score: 12
Game 8366 score: 14
Game 8367 score: 14
Game 8368 score: 14
Game 8369 score: 13
Game 8370 score: 15
Game 8371 score: 13
Game 8372 score: 16
Game 8373 score: 14
Game 8374 score: 14
Game 8375 score: 12
Gam

Game 8739 score: 14
Game 8740 score: 13
Game 8741 score: 15
Game 8742 score: 13
Game 8743 score: 13
Game 8744 score: 13
Game 8745 score: 15
Game 8746 score: 15
Game 8747 score: 16
Game 8748 score: 13
Game 8749 score: 15
Game 8750 score: 12
Game 8751 score: 15
Game 8752 score: 11
Game 8753 score: 14
Game 8754 score: 13
Game 8755 score: 18
Game 8756 score: 15
Game 8757 score: 16
Game 8758 score: 13
Game 8759 score: 13
Game 8760 score: 17
Game 8761 score: 13
Game 8762 score: 14
Game 8763 score: 15
Game 8764 score: 17
Game 8765 score: 16
Game 8766 score: 9
Game 8767 score: 15
Game 8768 score: 14
Game 8769 score: 13
Game 8770 score: 13
Game 8771 score: 10
Game 8772 score: 14
Game 8773 score: 17
Game 8774 score: 13
Game 8775 score: 13
Game 8776 score: 12
Game 8777 score: 17
Game 8778 score: 10
Game 8779 score: 15
Game 8780 score: 13
Game 8781 score: 13
Game 8782 score: 15
Game 8783 score: 12
Game 8784 score: 15
Game 8785 score: 13
Game 8786 score: 18
Game 8787 score: 12
Game 8788 score: 13
G

Game 9155 score: 12
Game 9156 score: 15
Game 9157 score: 16
Game 9158 score: 14
Game 9159 score: 18
Game 9160 score: 15
Game 9161 score: 12
Game 9162 score: 14
Game 9163 score: 16
Game 9164 score: 13
Game 9165 score: 14
Game 9166 score: 13
Game 9167 score: 17
Game 9168 score: 11
Game 9169 score: 13
Game 9170 score: 15
Game 9171 score: 14
Game 9172 score: 13
Game 9173 score: 4
Game 9174 score: 13
Game 9175 score: 15
Game 9176 score: 14
Game 9177 score: 15
Game 9178 score: 14
Game 9179 score: 14
Game 9180 score: 12
Game 9181 score: 14
Game 9182 score: 17
Game 9183 score: 14
Game 9184 score: 10
Game 9185 score: 12
Game 9186 score: 13
Game 9187 score: 12
Game 9188 score: 12
Game 9189 score: 15
Game 9190 score: 14
Game 9191 score: 15
Game 9192 score: 14
Game 9193 score: 12
Game 9194 score: 13
Game 9195 score: 14
Game 9196 score: 16
Game 9197 score: 11
Game 9198 score: 11
Game 9199 score: 13
Game 9200 score: 14
Game 9201 score: 12
Game 9202 score: 16
Game 9203 score: 8
Game 9204 score: 18
Ga

Game 9571 score: 13
Game 9572 score: 12
Game 9573 score: 12
Game 9574 score: 15
Game 9575 score: 8
Game 9576 score: 13
Game 9577 score: 12
Game 9578 score: 10
Game 9579 score: 14
Game 9580 score: 14
Game 9581 score: 9
Game 9582 score: 17
Game 9583 score: 15
Game 9584 score: 14
Game 9585 score: 7
Game 9586 score: 13
Game 9587 score: 14
Game 9588 score: 16
Game 9589 score: 15
Game 9590 score: 13
Game 9591 score: 17
Game 9592 score: 16
Game 9593 score: 11
Game 9594 score: 15
Game 9595 score: 15
Game 9596 score: 17
Game 9597 score: 17
Game 9598 score: 15
Game 9599 score: 13
Game 9600 score: 14
Game 9601 score: 13
Game 9602 score: 16
Game 9603 score: 13
Game 9604 score: 10
Game 9605 score: 11
Game 9606 score: 12
Game 9607 score: 12
Game 9608 score: -3
Game 9609 score: 18
Game 9610 score: 13
Game 9611 score: 13
Game 9612 score: 14
Game 9613 score: 14
Game 9614 score: 15
Game 9615 score: 13
Game 9616 score: 10
Game 9617 score: 17
Game 9618 score: 13
Game 9619 score: 14
Game 9620 score: 13
Gam

Game 9983 score: 15
Game 9984 score: 14
Game 9985 score: 15
Game 9986 score: 15
Game 9987 score: 14
Game 9988 score: 13
Game 9989 score: 13
Game 9990 score: 16
Game 9991 score: 14
Game 9992 score: 15
Game 9993 score: 15
Game 9994 score: 17
Game 9995 score: 16
Game 9996 score: 16
Game 9997 score: 13
Game 9998 score: 14
Game 9999 score: 15

Average score: 13.3613
Total time: 336.7969 seconds
Low Scores: {0: 5, 3: 12, 4: 25, 2: 12, 1: 13, 5: 46, -5: 1, -2: 4, -1: 3, -9: 1, -4: 2, -3: 1}


In [6]:
s = {'d':4}
t = (1, 2, 3)
s['d'] += 1
s

{'d': 5}